## Импорты

In [1]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')
import time
from pyspark.sql import SparkSession, DataFrameWriter
import pyspark.sql.functions as F

from pyspark.sql.functions import isnan, isnull

In [2]:
# Создание спарк сессии
spark = SparkSession.builder.master("local").enableHiveSupport().appName("extract-transform").getOrCreate()
spark

In [3]:
# Читаем данные из паркета
df = spark.read.format("parquet").load('data_in/competition_data_only_target_pqt/')

In [5]:
df.printSchema()

root
 |-- region_name: string (nullable = true)
 |-- city_name: string (nullable = true)
 |-- url_host: string (nullable = true)
 |-- date: date (nullable = true)
 |-- part_of_day: string (nullable = true)
 |-- request_cnt: long (nullable = true)
 |-- user_id: long (nullable = true)



In [6]:
# Создаем локальное представление датафрейма, как sql таблицы mts
df.createOrReplaceTempView("mts")

In [7]:
target = spark.read.format("parquet").load('data_in/public_train.pqt')

In [9]:
target.printSchema()

root
 |-- age: double (nullable = true)
 |-- is_male: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [8]:
target.createOrReplaceTempView("target")

In [11]:
data_learn = spark.sql("with sub_1 as"
                      " (select user_id, max(request_cnt) as max_request_cnt, "
                      " round(avg(request_cnt), 3) as avg_request_cnt, count(request_cnt)as count_request_cnt "
                      " from mts group by user_id)"
                      " select user_id, age, is_male, max_request_cnt, avg_request_cnt, count_request_cnt"
                      " from target left join sub_1 using(user_id)")

In [12]:
data_learn.write.parquet(path="data_out/data_for_learn_parquet/", mode="overwrite")

In [24]:
# Создадим функцию которая в будет принимать датафрейм и добавлять его к данным для обучения
def join_data_to_date_lern(join_df):
#     читаем последнюю версию данных для обучения
    data_learn = spark.read.format("parquet").load("data_out/data_for_learn_parquet_last/")
    data_learn = data_learn.join(join_df, "user_id", 'left')
    data_learn.write.parquet(path="data_out/data_for_learn_parquet/", mode="overwrite")
    return print("Данные сохранены")    

## Создаем признаки на основе request_cnt

### Создаем признаки на основе request_cnt и агрегирующих функций макс, среднее, подсчет

In [7]:
%%time
spark.sql("select user_id, max(request_cnt) as max_request_cnt, "
          "round(avg(request_cnt), 3) as avg_request_cnt, count(request_cnt)as count_request_cnt "
          "from mts group by user_id").show(20)

+-------+---------------+---------------+-----------------+
|user_id|max_request_cnt|avg_request_cnt|count_request_cnt|
+-------+---------------+---------------+-----------------+
| 366198|              6|          2.352|             2017|
| 166581|             10|           1.77|             1424|
| 414978|              4|          1.351|               57|
| 158464|              6|          1.626|              878|
|  97699|              6|          1.676|              982|
|  79418|              6|           1.57|              790|
| 324572|              6|          1.745|             1381|
|  38510|              6|          1.654|              593|
| 102613|              4|          1.235|               98|
| 266206|              5|          1.287|              254|
| 299953|              6|          1.348|              417|
| 134319|              4|          1.627|              308|
| 116139|              6|          1.678|             1479|
| 152419|              6|          1.889

In [14]:
%%time
# Посчитаем агрегирующие функции по request_cnt для части суток ночь
df_new = spark.sql("select user_id, max(request_cnt) as max_night_request_cnt, "
          "round(avg(request_cnt), 3) as avg_night_request_cnt, count(request_cnt)as count_night_request_cnt "
          "from mts where part_of_day = 'night' group by user_id")

CPU times: user 0 ns, sys: 6.45 ms, total: 6.45 ms
Wall time: 78.8 ms


In [17]:
join_data_to_date_lern(df_new)

Данные сохранены


In [27]:
df_test = spark.read.format("parquet").load("data_out/data_for_learn_parquet/")

In [20]:
df_test.orderBy("user_id").show(50, vertical=True)

-RECORD 0------------------------
 user_id                 | 0     
 age                     | 35.0  
 is_male                 | 0     
 max_request_cnt         | 5     
 avg_request_cnt         | 1.473 
 count_request_cnt       | 131   
 max_night_request_cnt   | 1     
 avg_night_request_cnt   | 1.0   
 count_night_request_cnt | 1     
-RECORD 1------------------------
 user_id                 | 1     
 age                     | 41.0  
 is_male                 | 0     
 max_request_cnt         | 6     
 avg_request_cnt         | 1.496 
 count_request_cnt       | 700   
 max_night_request_cnt   | 3     
 avg_night_request_cnt   | 1.088 
 count_night_request_cnt | 34    
-RECORD 2------------------------
 user_id                 | 2     
 age                     | 25.0  
 is_male                 | 0     
 max_request_cnt         | 4     
 avg_request_cnt         | 1.154 
 count_request_cnt       | 356   
 max_night_request_cnt   | 1     
 avg_night_request_cnt   | 1.0   
 count_night_r

In [25]:
%%time
# Посчитаем агрегирующие функции по request_cnt для части суток день
df_new = spark.sql("select user_id, max(request_cnt) as max_day_request_cnt, "
          "round(avg(request_cnt), 3) as avg_day_request_cnt, count(request_cnt)as count_day_request_cnt "
          "from mts where part_of_day = 'day' group by user_id")

CPU times: user 1.36 ms, sys: 3.95 ms, total: 5.31 ms
Wall time: 37.6 ms


In [26]:
join_data_to_date_lern(df_new)

Данные сохранены


In [28]:
df_test.orderBy("user_id").show(50, vertical=True)

-RECORD 0------------------------
 user_id                 | 0     
 age                     | 35.0  
 is_male                 | 0     
 max_request_cnt         | 5     
 avg_request_cnt         | 1.473 
 count_request_cnt       | 131   
 max_night_request_cnt   | 1     
 avg_night_request_cnt   | 1.0   
 count_night_request_cnt | 1     
 max_day_request_cnt     | 5     
 avg_day_request_cnt     | 1.911 
 count_day_request_cnt   | 56    
-RECORD 1------------------------
 user_id                 | 1     
 age                     | 41.0  
 is_male                 | 0     
 max_request_cnt         | 6     
 avg_request_cnt         | 1.496 
 count_request_cnt       | 700   
 max_night_request_cnt   | 3     
 avg_night_request_cnt   | 1.088 
 count_night_request_cnt | 34    
 max_day_request_cnt     | 6     
 avg_day_request_cnt     | 1.599 
 count_day_request_cnt   | 227   
-RECORD 2------------------------
 user_id                 | 2     
 age                     | 25.0  
 is_male      

In [12]:
%%time
# Посчитаем агрегирующие функции по request_cnt для части суток утро
spark.sql("select user_id, max(request_cnt) as max_morning_request_cnt, "
          "round(avg(request_cnt), 3) as avg_morning_request_cnt, count(request_cnt)as count_morning_request_cnt "
          "from mts where part_of_day = 'morning' group by user_id").show(20)

+-------+-----------------------+-----------------------+-------------------------+
|user_id|max_morning_request_cnt|avg_morning_request_cnt|count_morning_request_cnt|
+-------+-----------------------+-----------------------+-------------------------+
| 366198|                      6|                  2.252|                      330|
| 166581|                      8|                  1.845|                      277|
| 414978|                      1|                    1.0|                       20|
| 158464|                      6|                  1.742|                      476|
|  97699|                      6|                  1.528|                      373|
|  79418|                      5|                  1.429|                      326|
| 324572|                      6|                  1.872|                      516|
|  38510|                      6|                  1.646|                      161|
| 102613|                      4|                  1.526|                   

In [14]:
%%time
# Посчитаем агрегирующие функции по request_cnt для части суток вечер
spark.sql("select user_id, max(request_cnt) as max_evening_request_cnt, "
          "round(avg(request_cnt), 3) as avg_evening_request_cnt, count(request_cnt)as count_evening_request_cnt "
          "from mts where part_of_day = 'evening' group by user_id").show(20)

+-------+-----------------------+-----------------------+-------------------------+
|user_id|max_evening_request_cnt|avg_evening_request_cnt|count_evening_request_cnt|
+-------+-----------------------+-----------------------+-------------------------+
| 366198|                      6|                  2.346|                      778|
| 166581|                      8|                  1.502|                      645|
| 414978|                      1|                    1.0|                        1|
| 158464|                      4|                  1.327|                      159|
|  97699|                      5|                   1.83|                      247|
|  79418|                      3|                  1.182|                      143|
| 324572|                      4|                  1.428|                      355|
|  38510|                      6|                  1.728|                      173|
| 102613|                      2|                  1.038|                   

### Создаем признаки на основе request_cnt и даты с помощью агрегирующих функций

In [18]:
%%time
# Посчитаем сумму request_cnt для каждого юзера по датам и посмотрим макс, мин и среднюю сумму для пользователя
spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_request_cnt, min(sum_date_request_cnt) as min_sum_date_request_cnt, "
          "round(avg(sum_date_request_cnt), 3) as avg_sum_date_request_cnt"
          " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts group by user_id, date) as t1"
          " group by user_id").show(20)

+-------+------------------------+------------------------+------------------------+
|user_id|max_sum_date_request_cnt|min_sum_date_request_cnt|avg_sum_date_request_cnt|
+-------+------------------------+------------------------+------------------------+
|     26|                     152|                      11|                    71.8|
|     29|                     163|                       4|                  39.491|
|    474|                      52|                       4|                  21.295|
|    964|                     218|                       8|                  94.711|
|   1677|                      78|                       2|                  23.143|
|   1697|                     104|                     104|                   104.0|
|   1806|                       4|                       4|                     4.0|
|   2214|                     178|                       2|                  48.324|
|   2250|                     265|                       4|      

In [7]:
%%time
# Посчитаем сумму request_cnt для каждого юзера по дате и времени суток day и посмотрим макс, мин и среднюю сумму для пользователя
spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_day_request_cnt, min(sum_date_request_cnt) as min_sum_date_day_request_cnt, "
          "round(avg(sum_date_request_cnt), 3) as avg_sum_date_day_request_cnt"
          " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'day' group by user_id, date) as t1"
          " group by user_id").show(20)

+-------+----------------------------+----------------------------+----------------------------+
|user_id|max_sum_date_day_request_cnt|min_sum_date_day_request_cnt|avg_sum_date_day_request_cnt|
+-------+----------------------------+----------------------------+----------------------------+
|     26|                          47|                           1|                       11.05|
|     29|                          60|                           1|                      16.745|
|    474|                          24|                           1|                       7.857|
|    964|                          86|                           8|                      27.972|
|   1677|                          28|                           6|                       14.25|
|   1697|                          23|                          23|                        23.0|
|   1806|                           2|                           2|                         2.0|
|   2214|                     

In [8]:
%%time
# Посчитаем сумму request_cnt для каждого юзера по дате и времени суток night и посмотрим макс, мин и среднюю сумму для пользователя
spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_night_request_cnt, min(sum_date_request_cnt) as min_sum_date_night_request_cnt, "
          "round(avg(sum_date_request_cnt), 3) as avg_sum_date_night_request_cnt"
          " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'night' group by user_id, date) as t1"
          " group by user_id").show(20)

+-------+------------------------------+------------------------------+------------------------------+
|user_id|max_sum_date_night_request_cnt|min_sum_date_night_request_cnt|avg_sum_date_night_request_cnt|
+-------+------------------------------+------------------------------+------------------------------+
| 366198|                            49|                             1|                        10.593|
| 313329|                            21|                             1|                         9.286|
| 328967|                            65|                             1|                         7.056|
| 182662|                            17|                             1|                           6.5|
|  60756|                            87|                             1|                         13.65|
| 299914|                            18|                             1|                         5.143|
| 348263|                            25|                             1|  

In [9]:
%%time
# Посчитаем сумму request_cnt для каждого юзера по дате и времени суток morning и посмотрим макс, мин и среднюю сумму для пользователя
spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_morning_request_cnt, min(sum_date_request_cnt) as min_sum_date_morning_request_cnt, "
          "round(avg(sum_date_request_cnt), 3) as avg_sum_date_morning_request_cnt"
          " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'morning' group by user_id, date) as t1"
          " group by user_id").show(20)

+-------+--------------------------------+--------------------------------+--------------------------------+
|user_id|max_sum_date_morning_request_cnt|min_sum_date_morning_request_cnt|avg_sum_date_morning_request_cnt|
+-------+--------------------------------+--------------------------------+--------------------------------+
|     26|                              66|                               1|                          24.813|
|     29|                              72|                               1|                          15.526|
|    474|                               6|                               2|                           5.167|
|    964|                              32|                               1|                           6.235|
|   1677|                               7|                               1|                           3.714|
|   1697|                              44|                              44|                            44.0|
|   1806|          

In [10]:
%%time
# Посчитаем сумму request_cnt для каждого юзера по дате и времени суток evening и посмотрим макс, мин и среднюю сумму для пользователя
spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_evening_request_cnt, min(sum_date_request_cnt) as min_sum_date_evening_request_cnt, "
          "round(avg(sum_date_request_cnt), 3) as avg_sum_date_evening_request_cnt"
          " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'evening' group by user_id, date) as t1"
          " group by user_id").show(20)

+-------+--------------------------------+--------------------------------+--------------------------------+
|user_id|max_sum_date_evening_request_cnt|min_sum_date_evening_request_cnt|avg_sum_date_evening_request_cnt|
+-------+--------------------------------+--------------------------------+--------------------------------+
|     26|                              69|                               1|                          33.684|
|     29|                              96|                               1|                          15.651|
|    474|                              27|                               4|                          12.167|
|    964|                              84|                               1|                          24.079|
|   1677|                              45|                              30|                            37.5|
|   1697|                              25|                              25|                            25.0|
|   2214|          

<span style="color:green">Беглым взглядом видно, что пользователи которые делают запросы днем сильно отличаются от тех кто делает запросы ночью </span>

### Создаем признаки на основе дат и интервалов между датами

In [8]:
%%time
# Сгурппируем данные по user_id и date и посчитаем количество дат для каждого пользователя 
spark.sql("select user_id, count(date) as count_date"
          " from (select user_id, date from mts group by user_id, date) as t1 group by user_id").show(20)

+-------+----------+
|user_id|count_date|
+-------+----------+
|     26|        20|
|     29|        55|
|    474|        61|
|    964|        38|
|   1677|         7|
|   1697|         1|
|   1806|         1|
|   2214|       139|
|   2250|       111|
|   2509|        93|
|   3091|        23|
|   4590|        37|
|   4823|        44|
|   4894|         7|
|   5409|        39|
|   5556|         8|
|   7225|        25|
|   7279|        41|
|   7747|         9|
|   8075|         9|
+-------+----------+
only showing top 20 rows

CPU times: user 6.87 ms, sys: 6.88 ms, total: 13.7 ms
Wall time: 40 s


In [9]:
%%time
# Сгурппируем данные по user_id и date и посчитаем количество дат для каждого пользователя по запросам сделанным day
spark.sql("select user_id, count(date) as count_day_date"
          " from (select user_id, date from mts where part_of_day = 'day' group by user_id, date) as t1 group by user_id").show(20)

+-------+--------------+
|user_id|count_day_date|
+-------+--------------+
|     26|            20|
|     29|            47|
|    474|            35|
|    964|            36|
|   1677|             4|
|   1697|             1|
|   1806|             1|
|   1950|             1|
|   2214|           120|
|   2250|           103|
|   2509|            73|
|   3091|            22|
|   4590|            37|
|   4823|            43|
|   4894|             5|
|   5409|            35|
|   5556|             8|
|   7225|            25|
|   7279|            34|
|   7747|             3|
+-------+--------------+
only showing top 20 rows

CPU times: user 7.89 ms, sys: 5.72 ms, total: 13.6 ms
Wall time: 36.6 s


In [10]:
%%time
# Сгурппируем данные по user_id и date и посчитаем количество дат для каждого пользователя по запросам сделанным night
spark.sql("select user_id, count(date) as count_night_date"
          " from (select user_id, date from mts where part_of_day = 'night' group by user_id, date) as t1 group by user_id").show(20)

+-------+----------------+
|user_id|count_night_date|
+-------+----------------+
| 366198|              54|
| 299914|              14|
| 313329|              70|
| 328967|              18|
| 182662|              20|
|  60756|              40|
| 348263|              91|
| 349779|              61|
| 255774|              40|
| 257546|              46|
| 214552|              22|
| 244062|              15|
|  38543|              33|
| 404033|             117|
| 143916|              54|
| 203689|              56|
| 411907|              27|
| 104258|              80|
| 255354|              33|
| 152419|              58|
+-------+----------------+
only showing top 20 rows

CPU times: user 6.73 ms, sys: 4.24 ms, total: 11 ms
Wall time: 19.3 s


In [11]:
%%time
# Сгурппируем данные по user_id и date и посчитаем количество дат для каждого пользователя по запросам сделанным morning
spark.sql("select user_id, count(date) as count_morning_date"
          " from (select user_id, date from mts where part_of_day = 'morning' group by user_id, date) as t1 group by user_id").show(20)

+-------+------------------+
|user_id|count_morning_date|
+-------+------------------+
|     26|                16|
|     29|                38|
|    474|                 6|
|    964|                34|
|   1677|                 7|
|   1697|                 1|
|   1806|                 1|
|   2214|                78|
|   2250|               102|
|   2509|                51|
|   3091|                22|
|   4590|                37|
|   4823|                21|
|   4894|                 6|
|   5409|                17|
|   5556|                 7|
|   7225|                25|
|   7279|                34|
|   7747|                 4|
|   8075|                 2|
+-------+------------------+
only showing top 20 rows

CPU times: user 7.17 ms, sys: 6.5 ms, total: 13.7 ms
Wall time: 36.1 s


In [12]:
%%time
# Сгурппируем данные по user_id и date и посчитаем количество дат для каждого пользователя по запросам сделанным evening
spark.sql("select user_id, count(date) as count_evening_date"
          " from (select user_id, date from mts where part_of_day = 'evening' group by user_id, date) as t1 group by user_id").show(20)

+-------+------------------+
|user_id|count_evening_date|
+-------+------------------+
|     26|                19|
|     29|                43|
|    474|                60|
|    964|                38|
|   1677|                 2|
|   1697|                 1|
|   2214|               130|
|   2250|                96|
|   2509|                74|
|   3091|                21|
|   4590|                37|
|   4823|                41|
|   4894|                 3|
|   5409|                13|
|   5556|                 5|
|   7225|                24|
|   7279|                35|
|   7747|                 3|
|   8075|                 4|
|   8440|                29|
+-------+------------------+
only showing top 20 rows

CPU times: user 7.59 ms, sys: 5.27 ms, total: 12.9 ms
Wall time: 34 s


In [23]:
%%time
# Сгурппируем данные по user_id, date и part_of_day, посчитаем part_of_day, те мы увидим, что определенный пользователь делал запросы днем,
# утром и вечером  = 3 или днем и утром = 2, затем агрегируем по user_id и получим среднее, мин и макс
spark.sql("select user_id, avg(count_part_of_day_date) as avg_count_part_of_day_date,"
          " max(count_part_of_day_date) as max_count_part_of_day_date, min(count_part_of_day_date) as min_count_part_of_day_date"
          " from (select user_id, date, count(part_of_day) as count_part_of_day_date"
          " from (select user_id, date, part_of_day from mts group by user_id, date, part_of_day) as t1"
         " group by user_id, date) as t2 group by user_id").show(20)

+-------+--------------------------+--------------------------+--------------------------+
|user_id|avg_count_part_of_day_date|max_count_part_of_day_date|min_count_part_of_day_date|
+-------+--------------------------+--------------------------+--------------------------+
|   7225|                      3.96|                         4|                         3|
|  13460|         3.276595744680851|                         4|                         1|
|  15057|        2.8392857142857144|                         4|                         1|
|  15173|        3.6444444444444444|                         4|                         1|
|  15375|         3.823529411764706|                         4|                         2|
|  25207|                      2.96|                         3|                         2|
|  27651|                   2.96875|                         4|                         1|
|  56242|        2.8275862068965516|                         4|                         1|

In [36]:
%%time
# Сгруппируем данные по user_id, date, чтобы посчитать интервалы между датами по каждому пользователю
# Поскольку мы знаем что в данных есть разрыв в 109 дней 10 пользователей попали в разрыв данных:
# 155670, 327408, 28719, 330397, 273762, 78276, 188466, 220333, 265327, 406073. Их надо расмотреть подробнее.
# Удалим lag_date >= 109 и из итоговой таблицы исчезают пользователи 273762, 265327 потомучто у них по 1 лагу больше 109
spark.sql("select user_id, avg(lag_date) as avg_lag_date, max(lag_date) as max_lag_date, min(lag_date) as min_lag_date"
        " from (select user_id, int(date - lag(date) over (partition by user_id order by date)) as lag_date"
        " from (select user_id, date from mts group by user_id, date order by user_id, date) as t1) as t2"
        # " where lag_date < 109"
        " group by user_id order by user_id").show(20)

+-------+------------------+------------+------------+
|user_id|      avg_lag_date|max_lag_date|min_lag_date|
+-------+------------------+------------+------------+
|      0|             1.125|           3|           1|
|      1|1.1111111111111112|           3|           1|
|      2| 1.163265306122449|           3|           1|
|      3|1.1428571428571428|           3|           1|
|      4| 2.210526315789474|          12|           1|
|      5|1.2181818181818183|          11|           1|
|      8|1.3181818181818181|           4|           1|
|     12|1.0303030303030303|           2|           1|
|     15|1.0416666666666667|           2|           1|
|     16|1.1587301587301588|           3|           1|
|     18|1.1935483870967742|           3|           1|
|     20|3.2058823529411766|           8|           1|
|     21|1.0087719298245614|           2|           1|
|     22|1.1428571428571428|           3|           1|
|     23|               1.0|           1|           1|
|     25| 

<span style="color:blue"> **Возможно стоит удалить пользователей 273762, 265327, данные о них явно отрывочны. А лучше все удалить 155670, 327408, 28719, 330397, 273762, 78276, 188466, 220333, 265327, 406073. Проверить при обучении модели**</span>

<span style="color:red"> **Надо посчитать сколько пользователь делал запросов в выходные и будни**</span>

### Создаем признаки на основе region_name, city_name и агрегирующих функций

In [ ]:
region_name city_name

In [39]:
%%time
# Сгурппируем данные по user_id и region_name, посчитаем из скольки регионов пользователь делал запросы
spark.sql("select user_id, count(region_name) as count_region_name"
          " from (select user_id, region_name from mts group by user_id, region_name) as t1"
         " group by user_id order by user_id").show(20)

+-------+-----------------+
|user_id|count_region_name|
+-------+-----------------+
|      0|                1|
|      1|                3|
|      2|                1|
|      3|                1|
|      4|                5|
|      5|                2|
|      8|                1|
|     12|                1|
|     15|                1|
|     16|                1|
|     18|                1|
|     20|                3|
|     21|                2|
|     22|                3|
|     23|                1|
|     25|                1|
|     26|                1|
|     28|                2|
|     29|                8|
|     30|                1|
+-------+-----------------+
only showing top 20 rows

CPU times: user 28.3 ms, sys: 7.75 ms, total: 36.1 ms
Wall time: 39.5 s


In [40]:
%%time
# Сгурппируем данные по user_id и city_name, посчитаем из скольки городов пользователь делал запросы
spark.sql("select user_id, count(city_name) as count_city_name"
          " from (select user_id, city_name from mts group by user_id, city_name) as t1"
         " group by user_id order by user_id").show(20)

+-------+---------------+
|user_id|count_city_name|
+-------+---------------+
|      0|              1|
|      1|              6|
|      2|              1|
|      3|              1|
|      4|              9|
|      5|              4|
|      8|              5|
|     12|              5|
|     15|              1|
|     16|              1|
|     18|              2|
|     20|              7|
|     21|              2|
|     22|              5|
|     23|              2|
|     25|              2|
|     26|              1|
|     28|              6|
|     29|             16|
|     30|              1|
+-------+---------------+
only showing top 20 rows

CPU times: user 11.2 ms, sys: 2.33 ms, total: 13.5 ms
Wall time: 36.5 s


**На модели катбуст лучше всего себя показали агрегированные с помощью среднего признаки, из выше приведенных**

In [ ]:
%%time
# Посчитаем по user_id и url_host сумму request_cnt (те мы получим сумму запросов request_cnt к определенном хосту для каждого
# пользователя). После чего проранжируем по убыванию суммы и выберем топ-1 хост для каждого пользователя по сумме запросов
spark.sql("select user_id, url_host as top_1_url_sum_request_cnt"
          " from (select user_id, url_host, ROW_NUMBER() over (partition by user_id order by sum_request_cnt desc) as rank_sum_request_cnt "
          " from (select user_id, url_host, sum(request_cnt) as sum_request_cnt"
          " from mts group by user_id, url_host) as t1) as t2"
          " where rank_sum_request_cnt = 1").show(20)
# Также получим топ-2 и топ-3 хосты

In [ ]:
%%time
# Посчитаем по user_id и url_host колисечтво записей с определенным url_host (те мы получим колисечтво записей с обращением 
# к определенном хосту для каждого пользователя). После чего проранжируем по убыванию колисечтва и выберем топ-1 хост 
# для каждого пользователя по колисечтво запросов
spark.sql("select user_id, url_host as top_1_url_count_request_cnt"
          " from (select user_id, url_host, ROW_NUMBER() over (partition by user_id order by count_url_host desc) as rank_count_url_host "
          " from (select user_id, url_host, count(url_host) as count_url_host"
          " from mts group by user_id, url_host) as t1) as t2"
          " where rank_count_url_host = 1").show(20)
# Также получим топ-2 и топ-3 хосты